In [1]:
import json
import string
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory


import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Load json file
f = open('intent/intent.json', 'r')
intent_json = json.load(f)

In [4]:
# Create list from json
input = []
intent = []

for i in range(len(intent_json['intents'])):
    for user_input in intent_json['intents'][i]['input']:
        input.append(user_input)
        intent.append(intent_json['intents'][i]['intent'])

In [5]:
# Define stemmer and stopword remover
factory = StemmerFactory()
stemmer = factory.create_stemmer()

factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()

In [6]:
stopwords.remove('ok')
stopwords.remove('oh')
stopwords.remove('tidak')
stopwords.remove('ya')

In [7]:
baku = pd.read_csv('lexicon/baku.csv')
std_word_replace = {}
for i, row in enumerate(baku['slang']):
    std_word_replace[row] = baku['baku'].iloc[i]

In [8]:
# Create text cleaning function
def clean_text(text):
    new_text = []
    text = text.lower() # Lowercase
    # Loop each word in a sentence
    for kata in text.split(): 
        # Keep word not in slang or standard word
        if kata not in std_word_replace: 
            new_text.append(kata) 
        # Replace non-formal word with standard word
        elif kata in std_word_replace:
            new_text+=std_word_replace[kata].split() 
    # Join words without stopwords after stemming
    new_text = ' '.join(
        stemmer.stem(word) for word in new_text if word not in stopwords
    )
    # Remove punctuations
    text = text.translate(str.maketrans('', '', string.punctuation))
    return new_text

In [10]:
kalimat = 'Aku laper banget gatau maunya makan jeruk apa lagi meniru-niru... daah ngelamar'
clean_text(kalimat)

'aku lapar banget tidak tahu mau makan jeruk apa tiru deh lamar'

In [11]:
with open("saved_model/encoder.pkl", "rb") as f:
    le = pickle.load(f)

In [12]:
# dict(sorted(x.items(), key=lambda item: item[1]))

ktp, pas foto, password masih ketuker2

In [13]:
le.classes_

array(['CV', 'biaya', 'daftar', 'dokumen', 'error upload', 'ktp',
       'link sosmed', 'lowongan', 'lupa password', 'nama', 'pas foto',
       'pengantar', 'penutup', 'qualification', 'responsibilities',
       'salary', 'sapa', 'sertifikat', 'skck', 'timeline', 'training',
       'transkrip-ijazah'], dtype=object)

# SPACY

In [14]:
import spacy
ner = spacy.load('model-best')

In [15]:
doc = ner('''
data scientist itu apa, data engineer, analis, data analyst, data analist, data science, DS, DA, DE science, analyst, engine, engineer
''')

In [16]:
spacy.displacy.render(doc, style='ent', jupyter=True)

In [17]:
for ent in doc.ents:
    print(ent.label_)

SCIENTIST
ENGINEER
ANALIS
ANALIS
ANALIS
SCIENTIST
SCIENTIST
ANALIS
SCIENTIST
ANALIS
ENGINEER


In [18]:
type(doc)

spacy.tokens.doc.Doc

In [19]:
k = doc.ents

In [20]:
k

(data scientist,
 data engineer,
 analis,
 data analyst,
 data analist,
 data science,
 DS,
 DA,
 DE science,
 analyst,
 engineer)

In [21]:
doc.ents[0].label_.lower()

'scientist'

In [22]:
import requests
import json

In [23]:
URL_backend = 'https://fiktifid-bot.herokuapp.com/'
data = {'user_input':'de?'}
r = requests.post(URL_backend, json=data)
resp = r.json()

In [24]:
data['user_input']

'de?'

In [25]:
dok = ner(data['user_input'])
dok.ents[0].label_

'ENGINEER'

In [26]:
label_idx = int(resp['prediction'])
if label_idx != 1000:
    i = 0
    while i < len(intent_json['intents']):
        if le.classes_[label_idx] == intent_json['intents'][i]['intent'] and (le.classes_[label_idx] != 'responsibilities' or le.classes_[label_idx] != 'qualification'):
            response = intent_json['intents'][i]['response']
            break
        elif le.classes_[label_idx] == 'responsibilities':
            if dok.ents[0].label_.lower() == "scientist":
                response = ["Berikut adalah tanggung jawab yang akan diberikan untuk posisi Data Scientist:\n- Merancang dan mengembangkan berbagai solusi Machine Learning dan Deep Learning untuk meningkatkan pengalaman pengguna bagi konsumen\n- Berkolaborasi dengan seluruh elemen bisnis dan bertanggung jawab untuk merencanakan solusi end-to-end berbasis data untuk menyelesaikan permasalahan bisnis\n- Menjadi thinking partner bagi stakeholder lain untuk memperbaiki alur perjalanan data dan penggunaannya dalam operasional perusahaan, misal merancang proses feedback-loop atau human-in-the-loop untuk meningkatkan performa model secara berkelanjutan"]
                break
            elif dok.ents[0].label_.lower()=='engineer':
                response = ["Berikut adalah tanggung jawab yang akan diberikan untuk posisi Data Engineer:\n- Membangun dan menjaga end-to-end data pipeline dari input dan output heterogen\n- Menangani dan mengelola data warehouse\n- Membantu tim mentransformasikan data (ETL) dan mengembangkan proses ETL dari beberapa sumber\n- Menganalisis dan mengorganisir data mentah \n- Memastikan kualitas data dan integrasi data"]
                break
            elif dok.ents[0].label_.lower()=='analis':
                response = ["Berikut adalah tanggung jawab yang akan diberikan untuk posisi Data Analyst:\n- Mengumpulkan dan menyediakan data untuk membantu stakeholder lain meningkatkan metrik bisnis perusahaan dan retensi pelanggan\n- Menganalisis data untuk menemukan insight yang dapat ditindaklanjuti seperti membuat funnel conversion analysis, cohort analysis, long-term trends, user segmentation, dan dapat membantu meningkatkan kinerja perusahaan dan mendukung pengambilan keputusan yang lebih baik\n - Mengidentifikasi kebutuhan dan peluang bisnis berdasarkan data yang tersedia"]
                break
        elif le.classes_[label_idx] == 'qualification':
            if dok.ents[0].label_.lower()=='scientist':
                response = ["Untuk posisi Data Scientist ada beberapa kualifikasi yang harus dipenuhi:\n1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan\n2. Memiliki pemahaman mendasar tentang Statistika Analitik, Machine Learning, Deep Learning untuk menyelesaikan permasalahan bisnis\n3. Memiliki pengalaman kerja di bidang Data Science selama 1-3 tahun\n4. Memiliki pemahaman dan pengalaman tentang Big Data\n5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik"]
                break
            elif dok.ents[0].label_.lower()=='engineer':
                response = ["Untuk posisi Data Engineer ada beberapa kualifikasi yang harus dipenuhi:\n1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan\n2. Memiliki pengalaman bekerja dengan tools untuk ETL seperti AWS Glue, SSIS, Informatica, dll.\n3. Memiliki pengalaman kerja di bidang Data Engineer selama 1-3 tahun\n4. Memiliki pemahaman yang baik tentang ETL, SQL, dan noSQL\n5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik"]
                break
            elif dok.ents[0].label_.lower()=='analis':
                response = ["Untuk posisi Data Analyst ada beberapa kualifikasi yang harus dipenuhi:\n1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan\n2. Memiliki pemahaman mendasar tentang Statistika Analitik dan Inferensial untuk mencari peluang bisnis\n3. Memiliki pengalaman kerja di bidang Data Analyst selama 1-3 tahun\n4. Memiliki pemahaman dan pengalaman tentang Big Data serta visualisasi dengan tools seperti Power BI, Tableau, dll.\n5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik"]
                break
        else:
            i+=1
else:
    response = ['Maaf, Kak. Aku tidak mengerti chatnya...']
print(response[0])

Maaf, Kak. Aku tidak mengerti chatnya...


In [37]:
user_input = 'kualifikasi'
clean_input = clean_text(user_input)
data = {'user_input': clean_input}
r = requests.post(URL_backend, json=data)
resp = r.json()
label_idx = int(resp['prediction'])
print(le.classes_[label_idx])

qualification


In [38]:
try:
    dok.ents[0].label_.lower()
except:
    dok = ner('ds')
print(dok.ents[0].label_.lower())

scientist


In [ ]:
try:
    try:
        dok = ner(data['user_input'])
        dok.ents[0].label_.lower()
    except:
        dok = ner('ds')
    if label_idx != 1000:
        i = 0
        while i < len(intent_json['intents']):
            if le.classes_[label_idx] == intent_json['intents'][i]['intent'] and (le.classes_[label_idx] != 'responsibilities' or le.classes_[label_idx] != 'qualification'):
                response = intent_json['intents'][i]['response']
                break
            elif le.classes_[label_idx] == 'responsibilities':
                if dok.ents[0].label_.lower() == "scientist":
                    response = ["Berikut adalah tanggung jawab yang akan diberikan untuk posisi Data Scientist:\n- Merancang dan mengembangkan berbagai solusi Machine Learning dan Deep Learning untuk meningkatkan pengalaman pengguna bagi konsumen\n- Berkolaborasi dengan seluruh elemen bisnis dan bertanggung jawab untuk merencanakan solusi end-to-end berbasis data untuk menyelesaikan permasalahan bisnis\n- Menjadi thinking partner bagi stakeholder lain untuk memperbaiki alur perjalanan data dan penggunaannya dalam operasional perusahaan, misal merancang proses feedback-loop atau human-in-the-loop untuk meningkatkan performa model secara berkelanjutan"]
                    break
                elif dok.ents[0].label_.lower()=='engineer':
                    response = ["Berikut adalah tanggung jawab yang akan diberikan untuk posisi Data Engineer:\n- Membangun dan menjaga end-to-end data pipeline dari input dan output heterogen\n- Menangani dan mengelola data warehouse\n- Membantu tim mentransformasikan data (ETL) dan mengembangkan proses ETL dari beberapa sumber\n- Menganalisis dan mengorganisir data mentah \n- Memastikan kualitas data dan integrasi data"]
                    break
                elif dok.ents[0].label_.lower()=='analis':
                    response = ["Berikut adalah tanggung jawab yang akan diberikan untuk posisi Data Analyst:\n- Mengumpulkan dan menyediakan data untuk membantu stakeholder lain meningkatkan metrik bisnis perusahaan dan retensi pelanggan\n- Menganalisis data untuk menemukan insight yang dapat ditindaklanjuti seperti membuat funnel conversion analysis, cohort analysis, long-term trends, user segmentation, dan dapat membantu meningkatkan kinerja perusahaan dan mendukung pengambilan keputusan yang lebih baik\n - Mengidentifikasi kebutuhan dan peluang bisnis berdasarkan data yang tersedia"]
                    break
            elif le.classes_[label_idx] == 'qualification':
                dok = ner(data['user_input'])
                if dok.ents[0].label_.lower()=='scientist':
                    response = ["Untuk posisi Data Scientist ada beberapa kualifikasi yang harus dipenuhi:\n1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan\n2. Memiliki pemahaman mendasar tentang Statistika Analitik, Machine Learning, Deep Learning untuk menyelesaikan permasalahan bisnis\n3. Memiliki pengalaman kerja di bidang Data Science selama 1-3 tahun\n4. Memiliki pemahaman dan pengalaman tentang Big Data\n5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik"]
                    break
                elif dok.ents[0].label_.lower()=='engineer':
                    response = ["Untuk posisi Data Engineer ada beberapa kualifikasi yang harus dipenuhi:\n1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan\n2. Memiliki pengalaman bekerja dengan tools untuk ETL seperti AWS Glue, SSIS, Informatica, dll.\n3. Memiliki pengalaman kerja di bidang Data Engineer selama 1-3 tahun\n4. Memiliki pemahaman yang baik tentang ETL, SQL, dan noSQL\n5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik"]
                    break
                elif dok.ents[0].label_.lower()=='analis':
                    response = ["Untuk posisi Data Analyst ada beberapa kualifikasi yang harus dipenuhi:\n1. Memiliki gelar sarjana di bidang informatika, ilmu komputer, statistika, matematika, atau bidang lain yang berhubungan\n2. Memiliki pemahaman mendasar tentang Statistika Analitik dan Inferensial untuk mencari peluang bisnis\n3. Memiliki pengalaman kerja di bidang Data Analyst selama 1-3 tahun\n4. Memiliki pemahaman dan pengalaman tentang Big Data serta visualisasi dengan tools seperti Power BI, Tableau, dll.\n5. Memiliki kemampuan bekerja sama, kepemimpinan, dan problem solving yang baik"]
                    break
            else:
                i+=1
    else:
        response = ['Maaf, Kak. Aku tidak mengerti chatnya...']
except:
    print("error")
    response = ['Maaf, Kak. Aku tidak mengerti chatnya...\n\n\rTerjadi error']
print(intent_json['intents'][i]['intent'])